1. YOLOv4 사용 준비 과정

In [ ]:
#darknet clone
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

#make 파일 변경
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile #webcam 사용을 위한 부분
!make

#darknet에 대한 권한 변경
!chmod 755 darknet

In [ ]:
#이미 학습된 YOLOv4 weights 다운
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# detection 결과를 포함한 이미지를 띄우는 함수
def ImShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline

    image = cv2.imread(path)
    height, width = image.shape[:2]
    resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
    
    fig = plt.gcf()
    fig.set_size_inches(10,10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()

2. YOLOv4로 detection 진행 (잘 다운이 받아졌는지 확인)

In [ ]:
!cd /content

In [ ]:
#구글 검색으로 찾을 수 있는 이미지를 가져와서 보여주기
!wget -O test.jpg (이미지주소)
from IPython.display import Image, display
display(Image(filename='test.jpg'))

In [ ]:
#detect 진행
!./darknet detect cfg/yolov4.cfg yolov4.weights street_view.jpg -thresh 0.30 -dont_show

In [ ]:
#detection 결과 출력
ImShow('predictions.jpg')

3. roboflow의 data로 학습 진행하기

In [ ]:
#dataset 폴더 만들기
%cd /content
!mkdir dataset

In [ ]:
#드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#custom data 가져오기
!unzip /content/gdrive/MyDrive/file/img_ver2.zip -d /content/

In [ ]:
#dataset 폴더에 data 저장
%cd /content/dataset/
!curl -L "https://public.roboflow.com/ds/zWrN7uazks?key=7UNJh3Swd4" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
#image 개수 확인, 각 리스트에는 이미지의 경로가 저장
%cd /
from glob import glob
img_list1 = glob('/content/data1/*.txt')
print(len(img_list1))

In [ ]:
#train용 이미지들의 경로와 valid용 이미지들의 경로가 적혀있는 텍스트 파일 생성
from sklearn.model_selection import train_test_split

train_img_list,val_img_list = train_test_split(img_list1,test_size=0.1,random_state=2000)
print(len(train_img_list),len(val_img_list))

with open('/content/gdrive/MyDrive/data_ver2/train.txt','w') as f:
    f.write('\n'.join(train_img_list)+'\n')
with open('/content/gdrive/MyDrive/data_ver2/val.txt','w') as f:
    f.write('\n'.join(val_img_list)+'\n')

In [ ]:
#이동
%cd /content/dataset
!ls

In [ ]:
#class의 이름을 저장하는 파일 생성 후 저장
%cd /content/
!cat > custom.names

In [ ]:
#data 파일 생성 후 저장
!cat > custom.data

In [ ]:
!mkdir backup

yolov4-custom.cfg 파일 수정 진행

In [ ]:
%cd /content/darknet

In [ ]:
%cd /content/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
#콘솔 창에 입력하면 재연결 반복
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)

In [ ]:
#학습 시작
!./darknet detector train /content/gdrive/MyDrive/data_ver2/custom.data cfg/yolov4-custom.cfg /content/gdrive/MyDrive/data_ver2/backup/yolov4-custom_last.weights -dont_show -map

In [ ]:
#loss를 나타내는 그래프
ImShow('chart.png')

In [ ]:
#class에 대한 mAP등의 값을 보여줌
!./darknet detector map /content/gdrive/MyDrive/data_ver2/custom.data cfg/yolov4-custom.cfg /content/gdrive/MyDrive/data_ver2/backup/yolov4-custom_final.weights

* 얼마나 잘 학습되었는지 실제 사진 및 영상으로 확인

In [ ]:

%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-custom.cfg
%cd ..

In [ ]:
#test 폴더에 있던 사진으로 해보기
!./darknet detector test /content/gdrive/MyDrive/data_ver2/custom.data cfg/yolov4-custom.cfg /content/gdrive/MyDrive/data_ver2/backup/yolov4-custom_best.weights test.jpg -thresh 0.3
ImShow('predictions.jpg')

In [ ]:
%cd /content/darknet/
!./darknet detector demo /content/gdrive/MyDrive/data_ver2/custom.data cfg/yolov4-custom.cfg /content/gdrive/MyDrive/data_ver2/backup/yolov4-custom_best.weights test1.mp4 -thresh 0.3 -dont_show -out_filename test1_detect.mp4